In [277]:
import numpy as np

In [372]:
class Linear:
    def __init__(self):
        self.nn = []
        self.hiddenLayers = []
        self.outputLayers = []
        self.fcLayers = 0
    def addLayers(self,inputs,outputs):
        if self.nn:
            lastLayer = self.nn[-1]
            if lastLayer and lastLayer['output_neurons'] != inputs:
                raise Exception("number of input neurons should be equal to previous layer output neuron")
        param = self.getParams(inputs,outputs)
        self.fcLayers += 1
        layer = {'input_neurons':inputs,'output_neurons':outputs,'hidden_layer':param}
        self.nn.append(layer)
    
    def getParams(self,inputs,outputs):
        np.random.seed(100)
        params = {}
        for i in range(outputs):
            param = {} 
            for j in range(inputs):
                param['w' + str(i+1)] = np.random.randn(
                inputs, 1) * 0.1
                #param['x' + str(i+1)] = np.zeros((inputs,))
            params['n' + str(i+1)] = param
            #params['b' + str(index)] = np.random.randn(
            #outputs, 1) * 0.1
        return params
    
    def sigmoid(self,x):
        return np.exp(x)/(1+np.exp(x))
    
    def relu(x):
        x[x<0] = 0
        return x
    
    def activation(self,x,weight,activationMethod):
       # print('weight: ',weight)
        #print('inputs: ',x)
        if activationMethod == 'sigmoid':
            return self.sigmoid(np.dot(x,weight)).item()
        else:
            return self.relu(np.dot(weight,x))
        
    def forward(self,x,layerIndex):
        outputNeurons = []
        outputLayer = self.nn[layerIndex]
        inputLayer = self.nn[layerIndex-1]
        totalOutputNeurons = outputLayer['output_neurons']
        hiddenLayers = outputLayer['hidden_layer']
#         if layerIndex > 0:
#             print('input layer neurons: ',inputLayer['hidden_layer']['activations'])
        for i in range(totalOutputNeurons):
            activationVal = 0.0
            neurons = x if layerIndex == 0 else inputLayer['hidden_layer']['activations']
            weights = hiddenLayers['n'+str(i+1)]['w'+str(i+1)]
            activationVal = self.activation(neurons,weights,'sigmoid')
            outputNeurons.append(activationVal)
        self.nn[layerIndex]['hidden_layer']['activations'] = np.array(outputNeurons)
        #print('network: ',self.nn)
        
    def forwardPropagate(self,x):
        for index,fcLayer in enumerate(self.nn):
            self.forward(x,index)
        output_neuron = self.nn[-1] 
        return output_neuron['hidden_layer']['activations']
    
    def MSE(self,yPred,y):
        #print('prediction ',y_pred)
        #print('actual ',y)
        return 0.5*(y-yPred)**2
        #print(self.nn)
    def gradient(self,x,y,yPred,index):
        lossDerivativeYpred = yPred-y #upstream gradient dL/dyPred
        sigmoidDerivative = yPred * (1-yPred) #local gradient dyPred/dZ
        lossGradient = lossDerivativeYpred * sigmoidDerivative # dL/dZ. now this is the upstream gradient for next step
        inputNeurons = self.nn[index-1]['hidden_layer']
        currNeurons = self.nn[index]['hidden_layer']
        totalOutputNeurons = self.nn[index]['output_neurons']
        totalInputNeurons = self.nn[index]['input_neurons']
        neurons = inputNeurons['activations'] if index >0 else x # activation value of previous layer is the input x for the current layer
        for i in range(totalOutputNeurons):
            weights = currNeurons['n'+str(i+1)]['w'+str(i+1)]
            gradients = []
            for j in range(totalInputNeurons):
                gradient = neurons[j] * lossGradient
                gradients.append(gradient.item())
            self.nn[index]['hidden_layer']['n'+str(i+1)]['gradients'] = np.array(gradients)
        return gradients
        
    def backward(self,x,y,yPred):
        for i in reversed(range(len(self.nn))):
            self.gradient(x,y,yPred,i)
    def updateWeights(self,learningRate):
        for i in range(len(self.nn)):
            hiddenLayer = self.nn[i]['hidden_layer']
            totalOutputNeurons = self.nn[i]['output_neurons']
            totalInputNeurons = self.nn[i]['input_neurons']
            for j in range(totalOutputNeurons):
                weights = hiddenLayer['n'+str(j+1)]['w'+str(j+1)]
                gradients = hiddenLayer['n'+str(j+1)]['gradients']
                for k in range(totalInputNeurons):
                    #print('weight: ',self.nn[i]['hidden_layer']['n'+str(j+1)]['w'+str(j+1)][k])
                    #print('gradient: ',gradients[k])
                    self.nn[i]['hidden_layer']['n'+str(j+1)]['w'+str(j+1)][k] -= learningRate * gradients[k]

In [373]:
net = Linear()
net.addLayers(2,4)
net.addLayers(4,1)

In [391]:
data = np.array([[1.5, 2],[2.4,3.5],[4,3],[12,3]])
yData = np.array([[2],[4],[5],[8]])
xData = np.array([1.5, 2])

for epoch in range(10000):
    for index,x in enumerate(data):
        yPred = net.forwardPropagate(x)
        y = yData[index]
        loss = net.MSE(yPred,y)
        net.backward(x,y,yPred)
        #print('before update weight:', net.nn)
        net.updateWeights(0.1)
        #print('after update weight:', net.nn)
        #print('loss',loss)

        #print('loss ',loss)
print('network:', net.nn)

network: [{'input_neurons': 2, 'output_neurons': 4, 'hidden_layer': {'n1': {'w1': array([[21.11128898],
       [ 8.81522963]]), 'gradients': array([-0.00072739, -0.00018185])}, 'n2': {'w2': array([[21.01810337],
       [ 8.7334689 ]]), 'gradients': array([-0.00072739, -0.00018185])}, 'n3': {'w3': array([[20.95018271],
       [ 8.88398959]]), 'gradients': array([-0.00072739, -0.00018185])}, 'n4': {'w4': array([[21.06325748],
       [ 8.83003212]]), 'gradients': array([-0.00072739, -0.00018185])}, 'activations': array([1., 1., 1., 1.])}}, {'input_neurons': 4, 'output_neurons': 1, 'hidden_layer': {'n1': {'w1': array([[2.84069215],
       [2.97107245],
       [2.95849352],
       [2.88662024]]), 'gradients': array([-6.06154928e-05, -6.06154928e-05, -6.06154928e-05, -6.06154928e-05])}, 'activations': array([0.99999134])}}]


In [392]:
print(net.forwardPropagate(xData))

[0.99999134]
